In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%config Completer.use_jedi = False
import os
import time

import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
import statsmodels.api as sm
from joblib import Parallel, delayed
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor, Cursor
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from scipy.interpolate import InterpolatedUnivariateSpline, interp1d
from scipy.signal import argrelmax, argrelmin, find_peaks
from sklearn.linear_model import LinearRegression
from sidecar import Sidecar

os.chdir("..")
from src import definitions as defs
from src import features, models

In [2]:
wdir = defs.ROOT / "data"

In [3]:
freqstr = "30 min"
n_per_hour = pd.Timedelta("1 hour") / pd.Timedelta(freqstr)
data = pd.read_csv(wdir / "raw" / "mongla_tides.csv", index_col="datetime", parse_dates=True, infer_datetime_format=True)
data = data.reindex(pd.date_range(start=data.index[0], end=data.index[-1], freq=freqstr, name="datetime"))
data.elevation = data.elevation.interpolate(method="time", limit=1, limit_direction="both")
data["elapsed"] = data.index - data.index[0]
data = data[["elapsed", "elevation"]]

In [4]:
data[["high", "low"]] = False
distance = n_per_hour * 8
hi = find_peaks(x=data.elevation.values, distance=distance)[0]
lo = find_peaks(x=data.elevation.values * -1, distance=distance)[0]
data.loc[data.iloc[hi].index, "high"] = True
data.loc[data.iloc[lo].index, "low"] = True

In [5]:
window = pd.Timedelta("27h")
data["high_roll"] = data.elevation[data.high == True].rolling(window=window, center=True).mean().reindex(index=data.index).interpolate(method="time", limit_direction="both")
data["low_roll"] = data.elevation[data.low == True].rolling(window=window, center=True).mean().reindex(index=data.index).interpolate(method="time", limit_direction="both")
data["amp_roll"] = data.high_roll - data.low_roll

In [6]:
window = window
start = pd.to_datetime("1977-07-26")
end = start + pd.Timedelta("30D")

pdata = data.loc[start:end].reset_index()

fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()

sns.lineplot(data=pdata, x="datetime", y="elevation", color="cornflowerblue", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="high_roll", color="green", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="low_roll", color="red", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="amp_roll", color="black", alpha=0.3, ax=ax)

sns.scatterplot(data=pdata.loc[pdata.high == True], x="datetime", y="elevation", color="green", s=15, ax=ax, zorder=15)
sns.scatterplot(data=pdata.loc[pdata.low == True], x="datetime", y="elevation", color="red", s=15, ax=ax, zorder=15)

title = "Mongla Tidal Data from {} to {}".format(start.strftime("%b %Y"), end.strftime("%b %Y"))
xlabel = ""
ylabel = "Elevation (m)"
rotation = 0

locator = locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)

cursor = Cursor(ax=ax, useblit=True, linestyle="dashed")

ax.xaxis.set(major_locator=locator, major_formatter=formatter)
ax.tick_params(axis='x', labelrotation=rotation)
ax.set(xlabel=xlabel, ylabel=ylabel)
ax.set_title(label=title, pad=20)
ax.set_xlim(start, end)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
exog = (data.index - data.index[0]).total_seconds().astype(int)
frac = window * 3 / pd.Timedelta(freqstr) / len(data)
delta = window * 500 / pd.Timedelta(freqstr)
endogs = (data.high_roll.values, data.low_roll.values)
high_smooth, low_smooth = Parallel(n_jobs=2)(delayed(sm.nonparametric.lowess)(endog=endog, exog=exog, frac=frac, delta=delta, is_sorted=True) for endog in endogs)

In [8]:
data.loc[pd.TimedeltaIndex(list(zip(*high_smooth))[0], unit="s") + data.index[0], "high_smooth"] = list(zip(*high_smooth))[1]
data.loc[pd.TimedeltaIndex(list(zip(*low_smooth))[0], unit="s") + data.index[0], "low_smooth"] = list(zip(*low_smooth))[1]
data["amp_smooth"] = data.high_smooth - data.low_smooth

In [9]:
data[["spring", "neap"]] = False
distance = n_per_hour * 24 * 11 # search distance in hours
hi = find_peaks(x=data.amp_smooth.values, distance=distance)[0]
lo = find_peaks(x=data.amp_smooth.values * -1, distance=distance)[0]
data.loc[data.iloc[hi].index, "spring"] = True
data.loc[data.iloc[lo].index, "neap"] = True

In [22]:
window = window
start = pd.to_datetime("1998-07-15")
end = start + pd.Timedelta("30D")

pdata = data.loc[start:end].reset_index()

fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()

sns.lineplot(data=pdata, x="datetime", y="elevation", color="cornflowerblue", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="high_smooth", color="green", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="low_smooth", color="red", alpha=0.3, ax=ax)
sns.lineplot(data=pdata, x="datetime", y="amp_smooth", color="black", alpha=0.3, ax=ax)

sns.scatterplot(data=pdata.loc[pdata.high == True], x="datetime", y="elevation", color="green", s=15, ax=ax, zorder=15)
sns.scatterplot(data=pdata.loc[pdata.low == True], x="datetime", y="elevation", color="red", s=15, ax=ax, zorder=15)

for loc in pdata.loc[pdata.spring == True].datetime:
    ax.axvline(x=loc, color="black", linestyle="dotted", alpha=0.5)
    ax.text(x=loc, y=ax.get_ylim()[1], s="S", rotation=0, ha="center", va="bottom")

for loc in pdata.loc[pdata.neap == True].datetime:
    ax.axvline(x=loc, color="black", linestyle="dotted", alpha=0.5)
    ax.text(x=loc, y=ax.get_ylim()[1], s="N", rotation=0, ha="center", va="bottom")

title = "Mongla Tidal Data from {} to {}".format(start.strftime("%b %Y"), end.strftime("%b %Y"))
xlabel = ""
ylabel = "Elevation (m)"
rotation = 0

locator = locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)

# cursor = Cursor(ax=ax, useblit=True, linestyle="dashed")

ax.xaxis.set(major_locator=locator, major_formatter=formatter)
ax.tick_params(axis='x', labelrotation=rotation)
ax.set(xlabel=xlabel, ylabel=ylabel)
ax.set_title(label=title, pad=20)
ax.set_xlim(start, end)
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def lm_tides(data, ref_date, time_unit="Y"):

    x = ((data.index - ref_date) / np.timedelta64(1, time_unit)).values.reshape(-1, 1)
    y = data.values.reshape(-1, 1)
    lm = LinearRegression().fit(x, y)

    y_pred = lm.predict(x)

    x = np.array([data.index[0], data.index[-1]])
    y = np.array([y_pred[0, 0], y_pred[-1, 0]])
    s = pd.Series({"coef": lm.coef_[0, 0], "intercept": lm.intercept_[0], "x": x, "y": y})

    return(s)

In [13]:
def lm_tides(data, ref_date, time_unit="365.25 days"):

    x = ((data.index - ref_date) /  pd.Timedelta(time_unit)).values.reshape(-1, 1)
    y = data.values.reshape(-1, 1)
    lm = LinearRegression().fit(x, y)

    y_pred = lm.predict(x)

    arr1 = ["coef", "intercept", "TS", "TS"]
    arr2 = [pd.NaT, pd.NaT, data.index[0], data.index[-1]]

    index = pd.MultiIndex.from_arrays([arr1, arr2])

    s = pd.Series(data=(lm.coef_[0, 0], lm.intercept_[0], y_pred[0, 0], y_pred[-1, 0]), index=index)

    return(s)

In [15]:
daily = data.groupby(by=pd.Grouper(freq="D")).agg({"elevation": ["max", "min"]})
daily.columns = daily.columns.droplevel()
monthly = daily.groupby(by=pd.Grouper(freq="MS")).agg("mean").rename(columns={"max": "MHW", "min": "MLW"})

nodal_amp = 0.045
nodal_freq = 18.61

monthly["nodal"] = nodal_amp * np.sin(2*np.pi*(monthly.index - pd.to_datetime("2015-10-01")) / (pd.Timedelta("365.25 days") * nodal_freq) + np.pi / 2)
monthly["MHW_adj"] = monthly.MHW - monthly.nodal
monthly["MLW_adj"] = monthly.MLW - monthly.nodal

In [14]:
subset = monthly.loc["1998":"2009"]
subset_rates = subset.apply(lambda x: lm_tides(data=x, ref_date=subset.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()

sns.scatterplot(data=subset.reset_index(), x="datetime", y="MHW", color="green", s=15, ax=ax, zorder=15)
sns.scatterplot(data=subset.reset_index(), x="datetime", y="MLW", color="red", s=15, ax=ax, zorder=15)

sns.lineplot(data=subset_rates.MHW.TS, color="black", linestyle="dotted", ax=ax, zorder=1)
sns.lineplot(data=subset_rates.MLW.TS, color="black", linestyle="dotted", ax=ax, zorder=1)

ax.text(x=subset_rates.MHW.TS.index.mean(), y=subset_rates.MHW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW["coef"].values[0] * 1000), fontsize="medium")
ax.text(x=subset_rates.MLW.TS.index.mean(), y=subset_rates.MLW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW["coef"].values[0] * 1000), fontsize="medium")

ax.set(xlabel="", ylabel="Tide Elevation (m)")

title = "Mongla Mean Monthly Tidal Elevations\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
subset = monthly.loc["1998":"2014"]
subset_rates = subset.apply(lambda x: lm_tides(data=x, ref_date=subset.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(10, 5))
ax = fig.subplots()

sns.scatterplot(data=subset.reset_index(), x="datetime", y="MHW_adj", color="green", s=15, ax=ax, zorder=15)
sns.scatterplot(data=subset.reset_index(), x="datetime", y="MLW_adj", color="red", s=15, ax=ax, zorder=15)

sns.lineplot(data=subset_rates.MHW_adj.TS, color="black", linestyle="dotted", ax=ax, zorder=1)
sns.lineplot(data=subset_rates.MLW_adj.TS, color="black", linestyle="dotted", ax=ax, zorder=1)

ax.text(x=subset_rates.MHW_adj.TS.index.mean(), y=subset_rates.MHW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW_adj["coef"].values[0] * 1000), fontsize="medium")
ax.text(x=subset_rates.MLW_adj.TS.index.mean(), y=subset_rates.MLW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW_adj["coef"].values[0] * 1000), fontsize="medium")

ax.set(xlabel="", ylabel="Tide Elevation (m)")

title = "Mongla Mean Monthly Tidal Elevations\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
subset = monthly.loc["1998":"2009"]
subset_rates = subset.apply(lambda x: lm_tides(data=x, ref_date=subset.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(13, 10))
axs = fig.subplots(nrows=3, ncols=1, sharex=True)

# subplot 1

sns.scatterplot(data=subset.reset_index(), x="datetime", y="MHW", color="green", s=15, ax=axs[0], zorder=15)
sns.scatterplot(data=subset.reset_index(), x="datetime", y="MLW", color="red", s=15, ax=axs[0], zorder=15)

sns.lineplot(data=subset_rates.MHW.TS, color="black", linestyle="dotted", ax=axs[0], zorder=1)
sns.lineplot(data=subset_rates.MLW.TS, color="black", linestyle="dotted", ax=axs[0], zorder=1)

axs[0].text(x=subset_rates.MHW.TS.index.mean(), y=subset_rates.MHW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW["coef"].values[0] * 1000), fontsize="medium")
axs[0].text(x=subset_rates.MLW.TS.index.mean(), y=subset_rates.MLW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW["coef"].values[0] * 1000), fontsize="medium")

# subplot 2

sns.lineplot(data=subset.nodal, ax=axs[1], zorder=1)

# subplot 3 

sns.scatterplot(data=subset.reset_index(), x="datetime", y="MHW_adj", color="green", s=15, ax=axs[2], zorder=15)
sns.scatterplot(data=subset.reset_index(), x="datetime", y="MLW_adj", color="red", s=15, ax=axs[2], zorder=15)

sns.lineplot(data=subset_rates.MHW_adj.TS, color="black", linestyle="dotted", ax=axs[2], zorder=1)
sns.lineplot(data=subset_rates.MLW_adj.TS, color="black", linestyle="dotted", ax=axs[2], zorder=1)

axs[2].text(x=subset_rates.MHW_adj.TS.index.mean(), y=subset_rates.MHW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW_adj["coef"].values[0] * 1000), fontsize="medium")
axs[2].text(x=subset_rates.MLW_adj.TS.index.mean(), y=subset_rates.MLW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW_adj["coef"].values[0] * 1000), fontsize="medium")

axs[0].set(ylabel="Tide Elevation (m)")
axs[1].set(ylabel="Nodal Amplitude (m)")
axs[2].set(xlabel="", ylabel="Adjusted Tide Elevation (m)")

title = "Mongla Mean Monthly Tidal Elevations\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
PO = monthly.loc["1998":"2009"]
PO_rates = PO.apply(lambda x: lm_tides(data=x, ref_date=PO.index[0], time_unit="365.25 days"))

new = monthly.loc["1998":"2014"]
new_rates = new.apply(lambda x: lm_tides(data=x, ref_date=new.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(13, 10))
axs = fig.subplots(nrows=3, ncols=1, sharex=True)

# subplot 1

# sns.scatterplot(data=PO.reset_index(), x="datetime", y="MHW", color="green", alpha=0.3, s=15, ax=axs[0], zorder=15)
# sns.scatterplot(data=PO.reset_index(), x="datetime", y="MLW", color="red", alpha=0.3, s=15, ax=axs[0], zorder=15)

# sns.lineplot(data=PO_rates.MHW.TS, color="black", linestyle="dotted", alpha=0.3, ax=axs[0], zorder=1)
# sns.lineplot(data=PO_rates.MLW.TS, color="black", linestyle="dotted", alpha=0.3, ax=axs[0], zorder=1)

# axs[0].text(x=subset_rates.MHW.TS.index.mean(), y=subset_rates.MHW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW["coef"].values[0] * 1000), alpha=0.5, fontsize="medium")
# axs[0].text(x=subset_rates.MLW.TS.index.mean(), y=subset_rates.MLW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW["coef"].values[0] * 1000), alpha=0.5, fontsize="medium")


sns.scatterplot(data=new.reset_index(), x="datetime", y="MHW", color="green", alpha=0.5, s=15, ax=axs[0], zorder=15)
sns.scatterplot(data=new.reset_index(), x="datetime", y="MLW", color="red", alpha=0.5, s=15, ax=axs[0], zorder=15)

sns.lineplot(data=new_rates.MHW.TS, color="black", linestyle="dotted", ax=axs[0], zorder=1)
sns.lineplot(data=new_rates.MLW.TS, color="black", linestyle="dotted", ax=axs[0], zorder=1)

axs[0].text(x=new_rates.MHW.TS.index.mean(), y=new_rates.MHW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(new_rates.MHW["coef"].values[0] * 1000), fontsize="medium")
axs[0].text(x=new_rates.MLW.TS.index.mean(), y=new_rates.MLW.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(new_rates.MLW["coef"].values[0] * 1000), fontsize="medium")

# subplot 2

sns.lineplot(data=monthly.loc["1998":].nodal, ax=axs[1], zorder=1)

# subplot 3 

# sns.scatterplot(data=PO.reset_index(), x="datetime", y="MHW_adj", color="green", alpha=0.3, s=15, ax=axs[2], zorder=15)
# sns.scatterplot(data=PO.reset_index(), x="datetime", y="MLW_adj", color="red", alpha=0.3, s=15, ax=axs[2], zorder=15)

# sns.lineplot(data=PO_rates.MHW_adj.TS, color="black", linestyle="dotted", alpha=0.3, ax=axs[2], zorder=1)
# sns.lineplot(data=PO_rates.MLW_adj.TS, color="black", linestyle="dotted", alpha=0.3, ax=axs[2], zorder=1)

# axs[2].text(x=subset_rates.MHW_adj.TS.index.mean(), y=subset_rates.MHW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MHW_adj["coef"].values[0] * 1000),  alpha=0.5, fontsize="medium")
# axs[2].text(x=subset_rates.MLW_adj.TS.index.mean(), y=subset_rates.MLW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(subset_rates.MLW_adj["coef"].values[0] * 1000),  alpha=0.5, fontsize="medium")

sns.scatterplot(data=new.reset_index(), x="datetime", y="MHW_adj", color="green", alpha=0.5, s=15, ax=axs[2], zorder=15)
sns.scatterplot(data=new.reset_index(), x="datetime", y="MLW_adj", color="red", alpha=0.5, s=15, ax=axs[2], zorder=15)

sns.lineplot(data=new_rates.MHW_adj.TS, color="black", linestyle="dotted", ax=axs[2], zorder=1)
sns.lineplot(data=new_rates.MLW_adj.TS, color="black", linestyle="dotted", ax=axs[2], zorder=1)

axs[2].text(x=new_rates.MHW_adj.TS.index.mean(), y=new_rates.MHW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(new_rates.MHW_adj["coef"].values[0] * 1000), fontsize="medium")
axs[2].text(x=new_rates.MLW_adj.TS.index.mean(), y=new_rates.MLW_adj.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(new_rates.MLW_adj["coef"].values[0] * 1000), fontsize="medium")

axs[0].set(ylabel="Tide Elevation (m)")
axs[1].set(ylabel="Nodal Amplitude (m)")
axs[2].set(xlabel="", ylabel="Adjusted Tide Elevation (m)")

title = "Mongla Mean Monthly Tidal Elevations\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
subset = data.loc["1998":"2009"].copy()

springs = subset.loc[(subset.spring == True)][["high_smooth", "low_smooth"]]
springs["high_smooth"] = springs.high_smooth - nodal_amp * np.sin(2*np.pi*(springs.index - pd.to_datetime("2015-10-01")) / (pd.Timedelta("365.25 days") * nodal_freq) + np.pi / 2)
springs["low_smooth"] = springs.low_smooth - nodal_amp * np.sin(2*np.pi*(springs.index - pd.to_datetime("2015-10-01")) / (pd.Timedelta("365.25 days") * nodal_freq) + np.pi / 2)
spring_rates = springs.apply(lambda x: lm_tides(data=x, ref_date=subset.index[0], time_unit="365.25 days"))

neaps = subset.loc[(subset.neap == True)][["high_smooth", "low_smooth"]]
neaps["high_smooth"] = neaps.high_smooth - nodal_amp * np.sin(2*np.pi*(neaps.index - pd.to_datetime("2015-10-01")) / (pd.Timedelta("365.25 days") * nodal_freq) + np.pi / 2)
neaps["low_smooth"] = neaps.low_smooth - nodal_amp * np.sin(2*np.pi*(neaps.index - pd.to_datetime("2015-10-01")) / (pd.Timedelta("365.25 days") * nodal_freq) + np.pi / 2)
neap_rates = neaps.apply(lambda x: lm_tides(data=x, ref_date=subset.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(13, 5))
ax = fig.subplots()

sns.scatterplot(data=springs.reset_index(), x="datetime", y="high_smooth", color="green", marker=".", s=30, ax=ax, zorder=15)
sns.scatterplot(data=neaps.reset_index(), x="datetime", y="high_smooth", color="green", marker="+", s=20, ax=ax, zorder=15)

sns.lineplot(data=spring_rates.high_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)
sns.lineplot(data=neap_rates.high_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)

ax.text(x=spring_rates.high_smooth.TS.index.mean(), y=spring_rates.high_smooth.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(spring_rates.high_smooth["coef"].values[0] * 1000), fontsize="medium")
ax.text(x=neap_rates.high_smooth.TS.index.mean(), y=neap_rates.high_smooth.TS.max(), ha="center", va="bottom", zorder=20, s="${:+.1f} mm \, a^{{-1}}$".format(neap_rates.high_smooth["coef"].values[0] * 1000), fontsize="medium")

sns.scatterplot(data=springs.reset_index(), x="datetime", y="low_smooth", color="red", marker=".", s=30, ax=ax, zorder=15)
sns.scatterplot(data=neaps.reset_index(), x="datetime", y="low_smooth", color="red", marker="+", s=20, ax=ax, zorder=15)

sns.lineplot(data=spring_rates.low_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)
sns.lineplot(data=neap_rates.low_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)

ax.text(x=spring_rates.low_smooth.TS.index.mean(), y=spring_rates.low_smooth.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(spring_rates.low_smooth["coef"].values[0] * 1000), fontsize="medium")
ax.text(x=neap_rates.low_smooth.TS.index.mean(), y=neap_rates.low_smooth.TS.max(), ha="center", va="bottom", zorder=20, s="${:+.1f} mm \, a^{{-1}}$".format(neap_rates.low_smooth["coef"].values[0] * 1000), fontsize="medium")

ax.set(xlabel="", ylabel="Adjusted Tide Elevation (m)")
ax.set_ylim(0.5, 4.25)

title = "Spring vs Neap\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
subset = data.loc["1998":"2009"].copy()

springs = subset.loc[(subset.spring == True)].amp_smooth.to_frame()
spring_rates = springs.apply(lambda x: lm_tides(data=x, ref_date=x.index[0], time_unit="365.25 days"))

neaps = subset.loc[(subset.neap == True)].amp_smooth.to_frame()
neap_rates = neaps.apply(lambda x: lm_tides(data=x, ref_date=x.index[0], time_unit="365.25 days"))

fig = plt.figure(figsize=(13, 5))
ax = fig.subplots()

sns.lineplot(data=springs.reset_index(), x="datetime", y="amp_smooth", color="green", alpha=0.3, ax=ax, zorder=15, label="Spring")
sns.lineplot(data=neaps.reset_index(), x="datetime", y="amp_smooth", color="red", alpha=0.3, ax=ax, zorder=15, label="Neap")

sns.lineplot(data=spring_rates.amp_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)
sns.lineplot(data=neap_rates.amp_smooth.TS, color="black", linestyle="dotted", ax=ax, zorder=1)

ax.text(x=spring_rates.amp_smooth.TS.index.mean(), y=spring_rates.amp_smooth.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(spring_rates.amp_smooth["coef"].values[0] * 1000), fontsize="medium")
ax.text(x=neap_rates.amp_smooth.TS.index.mean(), y=neap_rates.amp_smooth.TS.max(), ha="center", va="bottom", zorder=30, s="${:+.1f} mm \, a^{{-1}}$".format(neap_rates.amp_smooth["coef"].values[0] * 1000), fontsize="medium")

ax.set(xlabel="", ylabel="Adjusted Tide Elevation (m)")
ax.set_ylim(1, 4.5)

title = "Spring vs Neap Amplitude\n({}-{})".format(subset.index[0].year, subset.index[-1].year)
fig.suptitle(t=title)

fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …